<a href="https://colab.research.google.com/github/Igprad01/Sentimen-Analisis/blob/main/Analisis_Sentimen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ANALISIS SENTIMEN CASE STUDY OPINI PUBLIK TWITTER

INPUT DATA SET

In [50]:
import pandas as pd

# load data
files = '/content/Gabungan sample crawling JOT jkt48.csv'
df = pd.read_csv(files)
df.head()
# display(df)


,conversation_id_str,created_at,favorite_count,full_text,id_str,image_url,in_reply_to_screen_name,lang,location,quote_count,reply_count,retweet_count,tweet_url,user_id_str,username
0,1.870000e+18,Mon Dec 30 18:55:01 +0000 2024,0,Om JOT Liat deh Si up girls ngajak @RFritzy_JK...,1.870000e+18,NaN,NaN,in,NaN,0,0,0,https://x.com/XJulianto/status/187380500346035...,1.820000e+18,XJulianto
1,1.870000e+18,Mon Dec 30 17:47:27 +0000 2024,0,@RFritzy_JKT48 Kalo oshinya JOT masih ga apa a...,1.870000e+18,https://pbs.twimg.com/media/GgEIPq1bYAExxgW.jpg,RFritzy_JKT48,in,"DKI Jakarta, Indonesia",0,0,0,https://x.com/Anggithaadr/status/1873788000431...,8.200000e+17,Anggithaadr
2,1.870000e+18,Mon Dec 30 15:52:30 +0000 2024,0,harapan untuk jkt48 yaitu membuat para jot sad...,1.870000e+18,NaN,NaN,in,NaN,0,0,0,https://x.com/blq_abilzz/status/18737590745775...,1.790000e+18,blq_abilzz
3,1.870000e+18,Mon Dec 30 15:25:31 +0000 2024,1,@_rybh @Maffi_f07 @Auwia_JKT48 Itu urusan inte...,1.870000e+18,NaN,_rybh,in,Salakanagara,0,1,0,https://x.com/gagitugie___/status/187375228321...,2.429397e+09,gagitugie___
4,1.870000e+18,Mon Dec 30 15:07:03 +0000 2024,0,@gagitugie___ @Maffi_f07 @Auwia_JKT48 Menunjuk...,1.870000e+18,NaN,gagitugie___,in,NaN,0,1,0,https://x.com/_rybh/status/1873747635750011333,3.181792e+09,_rybh


In [51]:
#tampil data
df_text = pd.DataFrame({
    'Original_Text': df['full_text'],
})

display(df_text)
#download
df_text.to_csv('df_text.csv', index=False)

,Original_Text
0,Om JOT Liat deh Si up girls ngajak @RFritzy_JK...
1,@RFritzy_JKT48 Kalo oshinya JOT masih ga apa a...
2,harapan untuk jkt48 yaitu membuat para jot sad...
3,@_rybh @Maffi_f07 @Auwia_JKT48 Itu urusan inte...
4,@gagitugie___ @Maffi_f07 @Auwia_JKT48 Menunjuk...
...,...
6211,Lahkk makasih om jot See u @Greesel_JKT48 http...
6212,@JambuBiji08 @Gracie_JKT48 Harusnya faham JOT om
6213,@greseella @Greesel_JKT48 Udah kena bedak JOT ...
6214,@naim_pasha @officialJKT48 @Y_ChikaJKT48 @F_Fe...


DETEKSI DUPLIKASI DATA

In [52]:
#duplicate data
df_duplicates = df[df.duplicated('full_text')]
#delete duplicates
df = df.drop_duplicates(subset='full_text')
display(df)
#download
df.to_csv('df_text_no_duplicates.csv', index=False)

,conversation_id_str,created_at,favorite_count,full_text,id_str,image_url,in_reply_to_screen_name,lang,location,quote_count,reply_count,retweet_count,tweet_url,user_id_str,username
0,1.870000e+18,Mon Dec 30 18:55:01 +0000 2024,0,Om JOT Liat deh Si up girls ngajak @RFritzy_JK...,1.870000e+18,NaN,NaN,in,NaN,0,0,0,https://x.com/XJulianto/status/187380500346035...,1.820000e+18,XJulianto
1,1.870000e+18,Mon Dec 30 17:47:27 +0000 2024,0,@RFritzy_JKT48 Kalo oshinya JOT masih ga apa a...,1.870000e+18,https://pbs.twimg.com/media/GgEIPq1bYAExxgW.jpg,RFritzy_JKT48,in,"DKI Jakarta, Indonesia",0,0,0,https://x.com/Anggithaadr/status/1873788000431...,8.200000e+17,Anggithaadr
2,1.870000e+18,Mon Dec 30 15:52:30 +0000 2024,0,harapan untuk jkt48 yaitu membuat para jot sad...,1.870000e+18,NaN,NaN,in,NaN,0,0,0,https://x.com/blq_abilzz/status/18737590745775...,1.790000e+18,blq_abilzz
3,1.870000e+18,Mon Dec 30 15:25:31 +0000 2024,1,@_rybh @Maffi_f07 @Auwia_JKT48 Itu urusan inte...,1.870000e+18,NaN,_rybh,in,Salakanagara,0,1,0,https://x.com/gagitugie___/status/187375228321...,2.429397e+09,gagitugie___
4,1.870000e+18,Mon Dec 30 15:07:03 +0000 2024,0,@gagitugie___ @Maffi_f07 @Auwia_JKT48 Menunjuk...,1.870000e+18,NaN,gagitugie___,in,NaN,0,1,0,https://x.com/_rybh/status/1873747635750011333,3.181792e+09,_rybh
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5504,1.750000e+18,Fri Jan 19 11:29:04 +0000 2024,1,Lahkk makasih om jot See u @Greesel_JKT48 http...,1.750000e+18,https://pbs.twimg.com/media/GEM7jzMasAApT7x.jpg,NaN,in,Indonesia,0,0,0,https://x.com/R_Hafidzmuh/status/1748306574794...,1.210000e+18,R_Hafidzmuh
5505,1.750000e+18,Fri Jan 19 11:14:00 +0000 2024,1,@JambuBiji08 @Gracie_JKT48 Harusnya faham JOT om,1.750000e+18,NaN,JambuBiji08,in,NaN,0,1,0,https://x.com/zibarrr_/status/1748302786700439566,1.600000e+18,zibarrr_
5506,1.750000e+18,Fri Jan 19 08:35:25 +0000 2024,0,@greseella @Greesel_JKT48 Udah kena bedak JOT ...,1.750000e+18,NaN,NaN,in,NaN,0,1,0,https://x.com/nouesophine/status/1748262876828...,1.740000e+18,nouesophine
5507,1.750000e+18,Fri Jan 19 02:11:57 +0000 2024,1,@naim_pasha @officialJKT48 @Y_ChikaJKT48 @F_Fe...,1.750000e+18,NaN,Naelep261,in,"DKI Jakarta, Indonesia",0,0,1,https://x.com/Naelep261/status/174816637217219...,1.290000e+18,Naelep261


PRE PROSESSING DATA

In [53]:
#case folding
df_text = pd.DataFrame({
    'Original_Text': df['full_text'],
    'Lowercased_Text': df['full_text'].str.lower(),
    # 'tweet_url': df['tweet_url']
})

display(df_text)
df_text.to_csv('df_text_case_folding.csv', index=False)

,Original_Text,Lowercased_Text
0,Om JOT Liat deh Si up girls ngajak @RFritzy_JK...,om jot liat deh si up girls ngajak @rfritzy_jk...
1,@RFritzy_JKT48 Kalo oshinya JOT masih ga apa a...,@rfritzy_jkt48 kalo oshinya jot masih ga apa a...
2,harapan untuk jkt48 yaitu membuat para jot sad...,harapan untuk jkt48 yaitu membuat para jot sad...
3,@_rybh @Maffi_f07 @Auwia_JKT48 Itu urusan inte...,@_rybh @maffi_f07 @auwia_jkt48 itu urusan inte...
4,@gagitugie___ @Maffi_f07 @Auwia_JKT48 Menunjuk...,@gagitugie___ @maffi_f07 @auwia_jkt48 menunjuk...
...,...,...
5504,Lahkk makasih om jot See u @Greesel_JKT48 http...,lahkk makasih om jot see u @greesel_jkt48 http...
5505,@JambuBiji08 @Gracie_JKT48 Harusnya faham JOT om,@jambubiji08 @gracie_jkt48 harusnya faham jot om
5506,@greseella @Greesel_JKT48 Udah kena bedak JOT ...,@greseella @greesel_jkt48 udah kena bedak jot ...
5507,@naim_pasha @officialJKT48 @Y_ChikaJKT48 @F_Fe...,@naim_pasha @officialjkt48 @y_chikajkt48 @f_fe...


In [54]:
#cleaning text (Sementara)
import re
def clean_text(text):
    text = str(text).lower()                                # lowercase
    text = re.sub(r'http\S+|www.\S+', '', text)             # hapus URL
    text = re.sub(r'<.*?>', '', text)                       # hapus tag HTML
    text = re.sub(r'@\w+', '', text)                        # hapus mention
    text = re.sub(r'#(\w+)', r'\1', text)                   # simpan kata dari hashtag
    text = re.sub(r'[-_]', ' ', text)                       # ubah - dan _ menjadi spasi
    text = re.sub(r'\d+', '', text)                         # hapus angka
    text = re.sub(r'[²³¹⁰⁴⁵⁶⁷⁸⁹⁺⁻⁼⁽⁾]', '', text)           # hapus karakter superscript
    text = re.sub(r'[^\w\s\u263a-\U0001f645]', '', text)    # hapus simbol kecuali emoji
    text = re.sub(r'\s+', ' ', text).strip()                # hapus spasi berlebih
    return text


df_text['Cleaned_Text'] = df_text['Lowercased_Text'].apply(clean_text)
display(df_text[['Lowercased_Text','Cleaned_Text']])
#download
df_text.to_csv('df_text_cleaning.csv', index=False)

,Lowercased_Text,Cleaned_Text
0,om jot liat deh si up girls ngajak @rfritzy_jk...,om jot liat deh si up girls ngajak supaya populer
1,@rfritzy_jkt48 kalo oshinya jot masih ga apa a...,kalo oshinya jot masih ga apa apa kan nder hehe
2,harapan untuk jkt48 yaitu membuat para jot sad...,harapan untuk jkt yaitu membuat para jot sadar...
3,@_rybh @maffi_f07 @auwia_jkt48 itu urusan inte...,itu urusan internal jot bkn urusan fans gak mu...
4,@gagitugie___ @maffi_f07 @auwia_jkt48 menunjuk...,menunjukan kalo proses yang katanya ketat yang...
...,...,...
5504,lahkk makasih om jot see u @greesel_jkt48 http...,lahkk makasih om jot see u
5505,@jambubiji08 @gracie_jkt48 harusnya faham jot om,harusnya faham jot om
5506,@greseella @greesel_jkt48 udah kena bedak jot ...,udah kena bedak jot aja nih
5507,@naim_pasha @officialjkt48 @y_chikajkt48 @f_fe...,streaming ini juga bisa jadi variety income bu...


In [55]:
!pip install sastrawi
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from collections import Counter
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [56]:
#Tokenization
df_text['Tokenized_Text'] = df_text['Cleaned_Text'].apply(word_tokenize)
#display(df_text)
display(df_text[['Cleaned_Text','Tokenized_Text']])
df_text.to_csv('df_text_tokenized.csv', index=False)

,Cleaned_Text,Tokenized_Text
0,om jot liat deh si up girls ngajak supaya populer,"[om, jot, liat, deh, si, up, girls, ngajak, su..."
1,kalo oshinya jot masih ga apa apa kan nder hehe,"[kalo, oshinya, jot, masih, ga, apa, apa, kan,..."
2,harapan untuk jkt yaitu membuat para jot sadar...,"[harapan, untuk, jkt, yaitu, membuat, para, jo..."
3,itu urusan internal jot bkn urusan fans gak mu...,"[itu, urusan, internal, jot, bkn, urusan, fans..."
4,menunjukan kalo proses yang katanya ketat yang...,"[menunjukan, kalo, proses, yang, katanya, keta..."
...,...,...
5504,lahkk makasih om jot see u,"[lahkk, makasih, om, jot, see, u]"
5505,harusnya faham jot om,"[harusnya, faham, jot, om]"
5506,udah kena bedak jot aja nih,"[udah, kena, bedak, jot, aja, nih]"
5507,streaming ini juga bisa jadi variety income bu...,"[streaming, ini, juga, bisa, jadi, variety, in..."


In [57]:
# Langkah 1: Tokenisasi jika belum
df_text['Tokenized_Text'] = df_text['Cleaned_Text'].apply(word_tokenize)

# Langkah 2: Gabungkan semua token
all_tokens = [token for sublist in df_text['Tokenized_Text'] for token in sublist]

# Langkah 3: Hitung frekuensi token
token_freq = Counter(all_tokens)
df_token_freq = pd.DataFrame(token_freq.items(), columns=['token', 'count'])

# Fungsi deteksi slang
def is_slang(token):
    # Aturan: kata berimbuhan biasanya diawali awalan seperti di-, ke-, se-, me-, ber-, ter-
    imbuhan_pattern = r'^(di|ke|se|me|ber|ter)[a-z]+'
    if re.match(imbuhan_pattern, token):
        return False
    if len(token) <= 5:
        return True
    return False


In [58]:

# Terapkan fungsi ke token
df_token_freq['is_slang'] = df_token_freq['token'].apply(is_slang)

# Filter hanya slang
df_slang = df_token_freq[df_token_freq['is_slang'] == True].copy()

In [59]:
#example
slang_to_formal = {
    'pls': 'please',
    'u': 'you',
    'gtg': 'got to go',
    'omg': 'oh my god',
    'lol': 'laugh out loud',
    'bff': 'best friend forever',
    'idk': 'i do not know',
    'brb': 'be right back',
    'tbh': 'to be honest',
    'smh': 'shaking my head',
    'lmao': 'laughing my ass off',
    'fyi': 'for your information',
    'asap': 'as soon as possible',
    'yw': 'you’re welcome'
}

In [60]:
# Fungsi untuk mencari padanan formal slang
def get_formal_for_slang(token):
    return slang_to_formal.get(token.lower(), '')

# Terapkan penggantian formal untuk slang
df_slang['formal'] = df_slang['token'].apply(get_formal_for_slang)

# Pilih hanya kolom slang dan formal
df_slang_formal = df_slang[['token', 'formal']].rename(columns={'token': 'slang'})

# Simpan hasil
df_slang_formal.to_csv('slang_formal_extracted.csv', index=False)

# Tampilkan hasil awal
display(df_slang_formal.head(50))


,slang,formal
0,om,
1,jot,
2,liat,
3,deh,
4,si,
5,up,
6,girls,
10,kalo,
12,masih,
13,ga,


In [40]:
# #Kamus
# df_kamus_alay = pd.read_csv('/content/kamusalay.csv')
# alay_to_formal = dict(zip(df_kamus_alay['alay'], df_kamus_alay['formal']))


# #Slang to formal
# df_slang_formal['formal'] = pd.read_csv('/content/slang_to_formal_mapping.csv')['formal']
# slang_to_formal = dict(zip(df_slang_formal['slang'], df_slang_formal['formal']))

# #KBBI
# with open('/content/list_0.5.1.txt', 'r') as f:
#     kbbi = f.read().splitlines()

In [64]:
#normalisasi
# 1. Load tokenized file
df = pd.read_csv('/content/df_text_tokenized.csv')

# Kalau perlu (opsional): pastikan tidak ada spasi di nama kolom
df.columns = df.columns.str.strip()

# 2. Load kamus
df_kamus_alay = pd.read_csv('/content/kamusalay.csv')
alay_to_formal = dict(zip(df_kamus_alay['alay'], df_kamus_alay['formal']))

df_slang_formal = pd.read_csv('/content/slang_to_formal_mapping.csv')
slang_to_formal = dict(zip(df_slang_formal['slang'], df_slang_formal['formal']))

with open('/content/list_0.5.1.txt', 'r') as f:
    kbbi_words = set(f.read().splitlines())

# 3. Convert Tokenized_Text string back to list if needed (check format!)
import ast
df['Tokenized_Text'] = df['Tokenized_Text'].apply(ast.literal_eval)

# 4. Fungsi normalisasi
def normalize_tokens(tokens):
    normalized = []
    for token in tokens:  # langsung iterasi list
        token = token.lower()
        if token in alay_to_formal:
            token = alay_to_formal[token]
        if token in slang_to_formal:
            token = slang_to_formal[token]
        normalized.append(token if token in kbbi_words else token)
    return normalized

# 5. Terapkan fungsi
df['normalized_tokens'] = df['Tokenized_Text'].apply(normalize_tokens)

# 6. Simpan hasil
df.to_csv('/content/hasil_normalisasi.csv', index=False)

✅ Tokenisasi dan normalisasi selesai!


In [ ]:
#Stopwords
stop_words = set(stopwords.words('indonesian'))
df_text['Stopwords_removal'] = df_text['Tokenized_Text'].apply(lambda x: [word for word in x if word not in stop_words])
df_text.head()
#download
df_text.to_csv('df_text_stopwords_removal.csv', index=False)

In [ ]:
#Stemming
factory = StemmerFactory()
stemmer = factory.create_stemmer()
df_text['Stemming'] = df_text['Stopwords_removal'].apply(lambda x: [stemmer.stem(word) for word in x])
df_text.head()

KeyboardInterrupt: 

In [ ]:
#dopwnload
df_text.to_csv('df_text_cleaning.csv', index=False)

PELABELAN DATA

In [ ]:
# Pelabelan Data


In [ ]:
# extrasi fitur

In [ ]:
# klasifikasi sentimen

In [ ]:
# evaluasi hasil


In [ ]:
# visualisasi dan hasilnya